In [6]:
#ETL Pipeline for Sales Data

import pandas as pd
import sqlite3
import os

# Define file paths
input_file = "/Users/sripavanyenugu/Downloads/amazon.csv"
output_file = "transformed_sales_data.csv"
db_file = "sales_database.db"


In [9]:
# Step 1: Extract - Read CSV File
if os.path.exists(input_file):
    df = pd.read_csv(input_file)
    print("✅ Data Extracted Successfully")
else:
    raise FileNotFoundError(f"❌ File {input_file} not found!")


✅ Data Extracted Successfully


In [10]:
df = pd.read_csv(input_file)
print(df.head())

   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   
2  Computers&Accessories|Accessories&Peripherals|...             ₹199   
3  Computers&Accessories|Accessories&Peripherals|...             ₹329   
4  Computers&Accessories|Accessories&Peripherals|...             ₹154   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,269   
1         ₹349                 43%  

In [14]:
# Step 2: Transform
df["discounted_price"] = df["discounted_price"].astype(str).str.replace("₹", "").str.replace(",", "").astype(float)
df["actual_price"] = df["actual_price"].astype(str).str.replace("₹", "").str.replace(",", "").astype(float)
df["discount_percentage"] = df["discount_percentage"].astype(str).str.replace("%", "").astype(float)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["rating_count"] = df["rating_count"].astype(str).str.replace(",", "").astype(float)


# Extract main category
df["main_category"] = df["category"].apply(lambda x: x.split("|")[0] if isinstance(x, str) else x)

# Calculate discount amount
df["discount_amount"] = df["actual_price"] - df["discounted_price"]

In [17]:
# Step 3: Load - Save to SQLite Database
conn = sqlite3.connect("amazon_data.db")
df.to_sql("amazon_products", conn, if_exists="replace", index=False)
conn.close()

print("ETL Process Completed! Data saved to SQLite.")


ETL Process Completed! Data saved to SQLite.


In [18]:
print(df.head())  # Shows the first 5 rows of the transformed DataFrame


   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category  discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             399.0   
1  Computers&Accessories|Accessories&Peripherals|...             199.0   
2  Computers&Accessories|Accessories&Peripherals|...             199.0   
3  Computers&Accessories|Accessories&Peripherals|...             329.0   
4  Computers&Accessories|Accessories&Peripherals|...             154.0   

   actual_price  discount_percentage  rating  rating_count  \
0        1099.0                 64.0     4.2       24269.0   
1         349.0       

In [20]:
df.to_csv("/Users/sripavanyenugu/Documents/transformed_amazon_data.csv", index=False)
print("Transformed data saved as transformed_amazon_data.csv")


Transformed data saved as transformed_amazon_data.csv
